# Downloads

In [13]:
#!pip install sentencepiece

# Imports

In [14]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

# Setting up LLM

## Importing the .Env variable

In [15]:
HF_TOKEN=os.environ.get('HF_TOKEN')
huggingface_repo_id="mistralai/Mistral-7B-Instruct-v0.3"

keeping the `temperature` low makes is accurate and very concise in the answers

In [16]:
def load_llm(huggingface_repo_id):
    llm = HuggingFaceEndpoint(  
        repo_id=huggingface_repo_id, 
        temperature=0.5,
        huggingfacehub_api_token=HF_TOKEN,
        task='conversational'
    )
    return llm


# Connecting to LLM and creating a chain

## Setting the prompt template

In [17]:
custom_prompt_template = """
Use the pieces of information provided in the context to answer user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Don't provide anything out of the given context

Context: {context}
Question: {question}

Start the answer directly. No small talk please.
"""

In [18]:
def set_custom_prompt(custom_prompt_template):
    prompt= PromptTemplate(template=custom_prompt_template,
                          input_variables=["context", "question"])
    return prompt

## Loading the FAISS Database

In [19]:
embedding_model= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [20]:
FAISS_DB_PATH = 'FAISS Database/'

Set the `allow_dangerous_deserialization` parameter to `True` becuase we trust the `FAISS` database
and its contents inside it because we made it

In [21]:
db = FAISS.load_local(FAISS_DB_PATH,
                      embedding_model,
                      allow_dangerous_deserialization=True)

## Creating the QA Chain

In [22]:
chat_model = ChatHuggingFace(llm=load_llm(huggingface_repo_id))
qa_chain=RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k':3}),
    return_source_documents=True,
    chain_type_kwargs={'prompt':set_custom_prompt(custom_prompt_template)}
)

## Creating user input

In [23]:
user_query=input("Ask WLF ")
response=qa_chain.invoke({'query': user_query})
print("Result : ", response["result"])
print("Source Documents: ", response["source_documents"])

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models/mistralai/Mistral-7B-Instruct-v0.3?expand=inferenceProviderMapping (Request ID: Root=1-68a34157-1e6c5f5d0afc677561364b30;e953c398-1133-42f6-b605-9e9ecc94014a)

Invalid credentials in Authorization header